# Goedel-Prover-V2 — Codespaces CPU Demo (JupyterLab)

> Note: 8B model on CPU FP32 is heavy. Use a large Codespace (>=32GB RAM) and keep n/max_length small. For practical runs, generate in Colab GPU then compile/summarize here.

In [1]:
import sys, subprocess, json, os, pathlib
print(sys.version)
!python -V
!pip -V

3.12.2 | packaged by conda-forge | (main, Feb 16 2024, 20:54:21) [Clang 16.0.6 ]
Python 3.12.2
pip 25.2 from /opt/miniconda3/lib/python3.12/site-packages/pip (python 3.12)


In [2]:
# Minimal Python deps for CPU inference + compile/summarize
!pip install -U pip
!pip install jupyterlab ipywidgets
!pip install torch --index-url https://download.pytorch.org/whl/cpu
!pip install transformers accelerate tqdm pandas sentencepiece

  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 94.2 kB/s  0:02:58m0:00:0300:06
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 151.0 kB/s  0:00:16 eta 0:00:01
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44/44 [jupyterlab]pyterlab]pyter-server]
Looking in indexes: https://download.pytorch.org/whl/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 949.5 kB/s  0:00:11m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pandas]2m1/2 [pandas]


## (Optional) Hugging Face login
Only needed if the model repo requires auth.

In [3]:
# from huggingface_hub import login
# login(token="hf_...")
pass

## Prepare a tiny input set
Keep input extremely small for CPU demo.

In [4]:
from pathlib import Path
src = Path('dataset/test.jsonl')
dst = Path('dataset/test_small.jsonl')
if src.exists():
    with src.open('r') as fin, dst.open('w') as fout:
        line = fin.readline()
        if line:
            fout.write(line)
    print('Wrote 1-line sample to', dst)
else:
    print('Warning: dataset/test.jsonl not found. Please add a JSONL input.')

Wrote 1-line sample to dataset/test_small.jsonl


## CPU Inference (very slow; high RAM)
Uses `inference_cpu.py`. Reduce `--n` and `--max_length`.

In [5]:
!python inference_cpu.py \
        --model_path Goedel-LM/Goedel-Prover-V2-8B \
        --input_path dataset/test_small.jsonl \
        --output_dir results/codespaces_cpu \
        --n 1 \
        --max_length 256 \
        --temp 0.2 \
        --use_cpu

python: can't open file '/Users/ts21/dev/Goedel-Prover-V2/inference_cpu.py': [Errno 2] No such file or directory


## Install Lean toolchain (elan)
Build `mathlib4` to enable compilation via Lean REPL.

In [6]:
# Install elan (Lean toolchain manager) — non-interactive
!curl -sSf https://raw.githubusercontent.com/leanprover/elan/master/elan-init.sh | sh -s -- -y
# Note: Lean binaries are installed under ~/.elan/bin/

info: downloading installer
info: default toolchain set to 'stable'


In [7]:
# Build mathlib4 (can take a while on first run)
!bash -lc 'cd mathlib4 && ~/.elan/bin/lake build'

⢿ [870/4652] Running Mathlib.Order.WellFounded (+ 7 more)re) 7 more)ore)re))

OSError: [Errno 5] Input/output error

## Compile generated codes via Lean REPL
Limit CPU parallelism and proof timeout for stability.

In [ ]:
!PROOF_TIMEOUT=180 python src/compile.py \
+        --input_path results/codespaces_cpu/to_inference_codes.json \
+        --output_path results/codespaces_cpu/code_compilation_repl.json \
+        --cpu 2

## Summarize results

In [ ]:
!python src/summarize.py \
+        --input_path results/codespaces_cpu/code_compilation_repl.json \
+        --full_record_path results/codespaces_cpu/full_records.json \
+        --output_dir results/codespaces_cpu/summary

import json, os
meta_path = 'results/codespaces_cpu/summary/meta_summarize.json'
if os.path.exists(meta_path):
    print(json.dumps(json.load(open(meta_path)), indent=2, ensure_ascii=False))
else:
    print('meta_summarize.json not found')

### Notes
- If CPU OOM occurs during inference, reduce `--max_length`, `--n`, and input size.
- For practical usage: run inference on Colab GPU (quantized) and copy `to_inference_codes.json` + `full_records.json` here, then run compile/summarize cells only.
- You can adjust `PROOF_TIMEOUT` and `--cpu` to control compile parallelism and timeout.